### TCC 6 – individual - Tatiana Prado Santos Massoco

# Desenvolvimento de Modelo Preditivo de Risco ESG para Investimentos Sustentáveis

Fonte de daddos: https://www.kaggle.com/datasets/tunguz/environment-social-and-governance-data/data

## Objetivo do Projeto  
Desenvolver um modelo preditivo de risco ESG (Environmental, Social, Governance) para investimentos sustentáveis. O modelo visa identificar riscos associados a fatores ESG, auxiliando na tomada de decisões alinhadas à sustentabilidade.  

---

## Ferramentas Necessárias  
- **Linguagem de Programação:** Python.  
- **Bibliotecas:** pandas, numpy, seaborn, matplotlib, scikit-learn.  
- **Ambiente:** Jupyter Notebook, Anaconda.  
- **Controle de Versão:** Git e GitHub.  
- **Deploy:** Flask, Docker.  
- **Monitoramento:** Prometheus e Grafana.  

---

## Modelo Preditivo

## Passo 7: Desenvolvimento do Modelo Preditivo


**Seleção das Variáveis Alvo**

- Identificar variáveis representativas de cada pilar ESG (Ambiental, Social e Governança).  

**Preparação dos Dados**

- Dividir os dados em conjuntos de treino e teste para garantir uma avaliação consistente dos modelos.  

**Escolha e Treinamento de Algoritmos**

Testar diferentes algoritmos de aprendizado supervisionado, como:  
- **Regressão Linear:** Para variáveis contínuas, como emissões de CO₂.  
- **Random Forest ou Gradient Boosting:** Para modelos mais robustos e flexíveis.  
- **Regressão Logística ou Classificação:** Caso existam variáveis categóricas.  

**Ajuste de Hiperparâmetros**

- Usar técnicas como:  
  - **Busca em Grade (Grid Search):** Para testar combinações sistemáticas de hiperparâmetros.  
  - **Busca Aleatória (Random Search):** Para otimização mais rápida e eficiente.  

**Validação**

- Avaliar os modelos com métricas apropriadas:  
  - **Para Regressão:** R², MAE, MSE.  
  - **Para Classificação:** Acurácia, F1, ROC-AUC.  

**Seleção do Melhor Modelo**

- Escolher o modelo com o melhor desempenho para cada variável alvo com base nos resultados das validações.  

**Documentação**

- Registrar no notebook as decisões tomadas, os códigos implementados e os resultados obtidos durante o treinamento e validação.  


In [4]:
# Importar bibliotecas necessárias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# Caminho para o arquivo de dados limpo e normalizado
caminho_dados = "../data/ESGData_clean_final.csv"

# Carregar o dataset
dados = pd.read_csv(caminho_dados)

# Exibir as primeiras linhas para verificar o carregamento
print(dados.head())

  Country Name Country Code  \
0   Arab World          ARB   
1   Arab World          ARB   
2   Arab World          ARB   
3   Arab World          ARB   
4   Arab World          ARB   

                                      Indicator Name     Indicator Code  \
0  Adjusted savings: natural resources depletion ...  NY.ADJ.DRES.GN.ZS   
1  Adjusted savings: net forest depletion (% of GNI)  NY.ADJ.DFOR.GN.ZS   
2                 Agricultural land (% of land area)     AG.LND.AGRI.ZS   
3  Agriculture, forestry, and fishing, value adde...     NV.AGR.TOTL.ZS   
4             CO2 emissions (metric tons per capita)     EN.ATM.CO2E.PC   

        1961       1962       1963       1964       1965       1966  ...  \
0  19.156353  19.804722  20.520000  20.950959  21.033578  21.147132  ...   
1  19.156353  19.804722  20.520000  20.950959  21.033578  21.147132  ...   
2  27.835643  27.826564  27.845522  27.847925  27.866972  27.866171  ...   
3  19.156353  19.804722  20.520000  20.950959  21.033578  

### Análise do Dataset Carregado

Esse output mostra as primeiras cinco linhas do dataset carregado e permite confirmar que os dados foram importados corretamente. Abaixo, segue uma análise do que pude observar:

### Estrutura do Dataset

**Colunas principais**

- **Country Name:** Nome do país ou região.  
- **Country Code:** Código ISO de três letras do país ou região.  
- **Indicator Name:** Nome do indicador, que descreve a métrica ou dado (ex: emissões de CO2, área agrícola, etc).  
- **Indicator Code:** Código curto associado ao indicador (ex.: NY.ADJ.DRES.GN.ZS).  

**Anos**

- Cada coluna numérica (ex: 1961, 1962, etc) representa valores do indicador para um determinado ano.  
- Existem também colunas derivadas de transformações logarítmicas (`1970_log`, `1980_log`, etc).  

**Formato do Dataset**

- Contém 68 colunas, incluindo informações descritivas e valores numéricos para diversos anos.  

---

### Observações Relevantes

**Indicadores e transformações**

- Os valores originais dos anos estão listados, junto com transformações logarítmicas aplicadas para normalização de dados (`1970_log`, `1980_log`, etc.).  
- Isso indica que já foi realizado um pré-processamento para melhorar a análise estatística e a modelagem.  

**Consistência dos dados**

- Todas as colunas numéricas parecem preenchidas com valores (não há sinais evidentes de valores ausentes nas primeiras linhas).  

**Formatos numéricos**

- Algumas colunas, como `2019`, têm valores muito altos, o que pode refletir diferentes escalas entre os indicadores.  

---

### Próximos passos com base nesse Dataset

**Validação dos dados**

- Verificar a presença de valores ausentes ou inconsistências no restante do dataset.  
- Confirmar que as transformações (ex.: colunas `_log`) foram aplicadas corretamente.  

**Exploração e Análise Estatística**

- Gerar estatísticas descritivas para entender a dispersão e distribuição dos valores.  
- Visualizar os dados para identificar padrões, tendências e possíveis outliers.  

**Preparação Adicional**

- Certificar que os dados estejam prontos para a modelagem, incluindo normalização, remoção de redundâncias e seleção de indicadores relevantes.  


In [7]:
# Lista de variáveis alvo para análise
variaveis_alvo = [
    "CO2 emissions (metric tons per capita)",  # Pilar Ambiental
    "Life expectancy at birth, total (years)",  # Pilar Social
    "Gini index (World Bank estimate)"         # Pilar Governança
]

# Iterar sobre cada variável alvo
for target_indicator in variaveis_alvo:
    print(f"\nPreparando os dados para a variável alvo: {target_indicator}")
    
    # Filtrar o dataset para o indicador atual
    dados_filtrados = dados[dados["Indicator Name"] == target_indicator]
    
    # Definir as colunas que devem ser removidas
    colunas_removidas = ["Country Name", "Country Code", "Indicator Name", "Indicator Code"]
    
    # Escolher o ano a ser previsto
    target_column = "2010"  # Escolha o ano desejado para previsão
    
    # Separar recursos (X) e variável alvo (y)
    X = dados_filtrados.drop(columns=colunas_removidas + [target_column])
    y = dados_filtrados[target_column]
    
    # Dividir os dados em treino e teste
    X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Verificar os shapes das divisões
    print(f"Shape do conjunto de treino para {target_indicator}: {X_treino.shape}")
    print(f"Shape do conjunto de teste para {target_indicator}: {X_teste.shape}")




Preparando os dados para a variável alvo: CO2 emissions (metric tons per capita)
Shape do conjunto de treino para CO2 emissions (metric tons per capita): (166, 63)
Shape do conjunto de teste para CO2 emissions (metric tons per capita): (42, 63)

Preparando os dados para a variável alvo: Life expectancy at birth, total (years)
Shape do conjunto de treino para Life expectancy at birth, total (years): (182, 63)
Shape do conjunto de teste para Life expectancy at birth, total (years): (46, 63)

Preparando os dados para a variável alvo: Gini index (World Bank estimate)
Shape do conjunto de treino para Gini index (World Bank estimate): (3, 63)
Shape do conjunto de teste para Gini index (World Bank estimate): (1, 63)


### Análise dos resultados da preparação dos dados

Os resultados indicam que os conjuntos de dados para cada variável alvo foram preparados com sucesso, mas há algumas observações importantes a serem feitas:

**Observações:**

**Tamanho dos Conjuntos de Dados**  
- Para as variáveis **"CO2 emissions (metric tons per capita)"** e **"Life expectancy at birth, total (years)"**, os conjuntos de treino e teste têm tamanhos razoáveis para treinar e avaliar modelos preditivos.  
- No caso do **"Gini index (World Bank estimate)"**, o conjunto de dados é extremamente pequeno, com apenas 4 exemplos no total.  
  - Isso pode dificultar a modelagem e avaliação, pois há poucos dados disponíveis para aprendizado.  

**Dimensão das Variáveis (63 colunas)**  
- Cada conjunto de treino e teste possui 63 variáveis (anos), o que oferece uma boa quantidade de informações para o modelo.  

**Discrepância no Número de Dados**  
- A baixa quantidade de dados para o indicador de governança pode estar relacionada à disponibilidade limitada no dataset original.  


In [8]:
# Importar bibliotecas necessárias
from sklearn.model_selection import train_test_split

# Lista de indicadores por pilar ESG
indicadores_esg = [
    # Pilar Ambiental
    "CO2 emissions (metric tons per capita)",
    "Methane emissions (metric tons of CO2 equivalent per capita)",
    "Nitrous oxide emissions (metric tons of CO2 equivalent per capita)",
    "Energy use (kg of oil equivalent per capita)",
    "Fossil fuel energy consumption (% of total)",
    "Electricity production from coal sources (% of total)",
    "Forest area (% of land area)",
    "Adjusted savings: natural resources depletion (% of GNI)",
    
    # Pilar Social
    "Life expectancy at birth, total (years)",
    "Literacy rate, adult total (% of people ages 15 and above)",
    "School enrollment, primary and secondary (gross), gender parity index (GPI)",
    "Prevalence of overweight (% of adults)",
    "Unemployment, total (% of total labor force)",
    "Individuals using the Internet (% of population)",
    "Access to electricity (% of population)",
    
    # Pilar Governança
    "Gini index (World Bank estimate)",
    "Government expenditure on education, total (% of government expenditure)",
    "Ratio of female to male labor force participation rate (%)",
    "Patent applications, residents",
    "Income share held by lowest 20%",
]

# Iterar sobre os indicadores ESG
for target_column in indicadores_esg:
    if target_column in dados["Indicator Name"].values:
        print(f"\nPreparando os dados para a variável alvo: {target_column}")
        
        # Filtrar o dataset apenas para o indicador atual
        dados_filtrados = dados[dados["Indicator Name"] == target_column]
        
        # Separar recursos (X) e variável alvo (y)
        X = dados_filtrados.drop(columns=["Country Name", "Country Code", "Indicator Name", "Indicator Code"])
        y = dados_filtrados.iloc[:, 4:]  # Colunas de anos como alvo
        
        # Verificar se há dados suficientes para dividir
        if len(X) > 1:
            # Dividir os dados em treino e teste
            X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)
            
            # Exibir os shapes das divisões
            print(f"Shape do conjunto de treino para {target_column}: {X_treino.shape}")
            print(f"Shape do conjunto de teste para {target_column}: {X_teste.shape}")
        else:
            print(f"A variável {target_column} não possui dados suficientes para dividir.")


Preparando os dados para a variável alvo: CO2 emissions (metric tons per capita)
Shape do conjunto de treino para CO2 emissions (metric tons per capita): (166, 64)
Shape do conjunto de teste para CO2 emissions (metric tons per capita): (42, 64)

Preparando os dados para a variável alvo: Methane emissions (metric tons of CO2 equivalent per capita)
Shape do conjunto de treino para Methane emissions (metric tons of CO2 equivalent per capita): (185, 64)
Shape do conjunto de teste para Methane emissions (metric tons of CO2 equivalent per capita): (47, 64)

Preparando os dados para a variável alvo: Nitrous oxide emissions (metric tons of CO2 equivalent per capita)
Shape do conjunto de treino para Nitrous oxide emissions (metric tons of CO2 equivalent per capita): (186, 64)
Shape do conjunto de teste para Nitrous oxide emissions (metric tons of CO2 equivalent per capita): (47, 64)

Preparando os dados para a variável alvo: Energy use (kg of oil equivalent per capita)
Shape do conjunto de tre

### Análise dos resultados da divisão dos dados

Os resultados indicam que, para a maioria dos indicadores, foi possivel dividir os dados em conjuntos de treino e teste com uma quantidade razoável de exemplos. Contudo, há alguns indicadores que apresentam dados insuficientes, como:

**Indicadores com Dados Insuficientes**

- **Gini index (World Bank estimate):** Apenas 4 exemplos no total.  
- **Income share held by lowest 20%:** Apenas 4 exemplos no total.  
- **Government expenditure on education, total (% of government expenditure):** Apenas 14 exemplos no total.  

---

### Análise geral

**Indicadores com Dados Suficientes**

- A maioria dos indicadores possui dados suficientes para modelagem (treino e teste).  
  - Isso é essencial para garantir que o modelo aprenda padrões de forma eficaz.  
- Exemplos como **CO2 emissions (metric tons per capita)** e **Life expectancy at birth, total (years)** têm mais de 200 exemplos no total, garantindo uma boa base para modelagem.  

**Indicadores com Dados Escassos**

- Alguns indicadores têm menos de 20 exemplos no total, como **Income share held by lowest 20%** e **Gini index (World Bank estimate)**.
  - Esses indicadores podem ser problemáticos para modelagem devido ao risco de overfitting e à falta de representatividade.  

**Distribuição de Treino e Teste**

- O código foi bem-sucedido ao dividir os dados em 80% para treino e 20% para teste para a maioria dos casos.  

---

### Próximos Passos

**Focar nos Indicadores com Dados Suficientes**

- Priorizar indicadores como **CO2 emissions**, **Methane emissions**, **Life expectancy**, entre outros com amostras robustas, para as próximas etapas de modelagem.  

**Abordar Indicadores com Dados Escassos**

- Considerar exclusão desses indicadores da modelagem ou buscar fontes adicionais de dados.  
- Alternativamente, explorar métodos específicos, como:
  - **Modelos simplificados:** Regressão linear simples.  
  - **Técnicas de oversampling:** Como SMOTE, para tentar aumentar a representatividade.  

**Iniciar a Modelagem Preditiva**

- Configurar modelos de aprendizado de máquina para os indicadores escolhidos.  
- Comparar métricas de desempenho, como:
  - **Para Regressão:** MAE, RMSE, R².  
  - **Para Classificação:** Accuracy, Precision, Recall.  
